In [3]:
import warnings
warnings.filterwarnings("ignore")


In [2]:
import sys
sys.path.append("..\src")
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR
df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,11.0,15.0,26.0,8.0,9.0,7.0,3.0,1.0,0.0,3.0,...,11.0,7.0,4.0,3.0,4.0,9.0,19.0,2022-01-29,4,17.0
1,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,5.0,4.0,10.0,7.0,5.0,9.0,10.0,2022-01-30,4,9.0
2,0.0,1.0,0.0,0.0,1.0,1.0,1.0,3.0,2.0,3.0,...,8.0,7.0,8.0,5.0,5.0,10.0,0.0,2022-01-31,4,3.0
3,1.0,1.0,0.0,0.0,0.0,3.0,2.0,3.0,4.0,5.0,...,3.0,16.0,7.0,1.0,0.0,1.0,3.0,2022-02-01,4,3.0
4,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,1.0,2.0,...,3.0,8.0,3.0,0.0,4.0,4.0,3.0,2022-02-02,4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-27,199,0.0
88290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-28,199,0.0
88291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-29,199,0.0
88292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-30,199,0.0


In [3]:
from datetime import datetime
from data_split import  train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 6, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(32226, 674)
y_train.shape=(32226,)
X_test.shape=(56068, 674)
y_test.shape=(56068,)


In [1]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from model import get_pipeline





def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = TimeSeriesSplit(n_splits=2)
    scores = []
    
    # sort X_train by `pikup_hour` inplace
    # so the TimeSeriesSplit will split the data in a consistent way
    X_train.sort_values('pickup_hour', inplace=True)

    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
            # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [2]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

[I 2024-02-19 13:36:08,037] A new study created in memory with name: no-name-af15189c-2968-4439-b292-c0326959074d
[W 2024-02-19 13:36:08,044] Trial 0 failed with parameters: {'num_leaves': 201, 'feature_fraction': 0.6479475068052254, 'bagging_fraction': 0.5545467339716765, 'min_child_samples': 84} because of the following error: NameError("name 'X_train' is not defined").
Traceback (most recent call last):
  File "d:\ML System\Taxi_Demand_Predictor\.venv\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_2328\2445139992.py", line 33, in objective
    X_train.sort_values('pickup_hour', inplace=True)
NameError: name 'X_train' is not defined
[W 2024-02-19 13:36:08,068] Trial 0 failed with value None.


NameError: name 'X_train' is not defined

In [6]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 126, 'feature_fraction': 0.5004209875157843, 'bagging_fraction': 0.5736946105373395, 'min_child_samples': 32}


In [7]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.5004209875157843, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5004209875157843
[LightGBM] [Warning] bagging_fraction is set=0.5736946105373395, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5736946105373395
[LightGBM] [Warning] feature_fraction is set=0.5004209875157843, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5004209875157843
[LightGBM] [Warning] bagging_fraction is set=0.5736946105373395, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5736946105373395
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.199844 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 154527
[LightGBM] [Info] Number of data points in the train set: 32226, number of used features: 675
[LightGBM] [Info] Start training from scor

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x0000029040840160>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.5736946105373395,
                               feature_fraction=0.5004209875157843,
                               min_child_samples=32, num_leaves=126))])

In [8]:
predictions = pipeline.predict(X_test)
from sklearn.metrics import  mean_absolute_error
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

[LightGBM] [Warning] feature_fraction is set=0.5004209875157843, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5004209875157843
[LightGBM] [Warning] bagging_fraction is set=0.5736946105373395, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5736946105373395
test_mae=19.9848


In [9]:
from plot import plot_one_sample

plot_one_sample(
    example_id=54,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)

In [10]:
plot_one_sample(
    example_id=130,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)